In [1]:
import pandas as pd
import numpy as np

In [2]:
url = r"C:\MY WORLD\Projects\User Journey analysis\SQL\user journey.csv"
raw_data = pd.read_csv(url)

## Data Cleaning

In [3]:
raw_data.head()

,user_id,session_id,first_purchase_date,subscription_type,event_date,user_journey
0,1516,2980231,2023-01-27 12:16:44,Annualy,2022-12-11 17:42:46,Log in-Other-Log in-Log in-Log in-Log in-Log i...
1,1516,2980248,2023-01-27 12:16:44,Annualy,2022-12-11 17:44:57,Log in-Log in-Log in-Log in-Log in-Log in-Sign...
2,1516,2992252,2023-01-27 12:16:44,Annualy,2022-12-12 13:22:59,Log in-Log in-Log in-Log in-Log in-Log in
3,1516,3070491,2023-01-27 12:16:44,Annualy,2022-12-15 08:24:45,Log in-Log in-Log in-Log in-Log in-Log in-Log ...
4,1516,3709807,2023-01-27 12:16:44,Annualy,2023-01-26 07:58:16,Log in-Log in-Log in-Log in-Log in-Log in-Log ...


In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6876 entries, 0 to 6875
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              6876 non-null   int64 
 1   session_id           6876 non-null   int64 
 2   first_purchase_date  6876 non-null   object
 3   subscription_type    6876 non-null   object
 4   event_date           6876 non-null   object
 5   user_journey         6876 non-null   object
dtypes: int64(2), object(4)
memory usage: 322.4+ KB


In [5]:
#Creating a copy of the raw data frame, which is to be modified, avoiding corrupting the original source of data. 
#Columns deleted are of no use for the analysis given the kinda narrow scope of the provided data

clean = raw_data.copy().drop(['session_id', 'first_purchase_date', 'event_date'], axis=1)


In [6]:
clean.head()

,user_id,subscription_type,user_journey
0,1516,Annualy,Log in-Other-Log in-Log in-Log in-Log in-Log i...
1,1516,Annualy,Log in-Log in-Log in-Log in-Log in-Log in-Sign...
2,1516,Annualy,Log in-Log in-Log in-Log in-Log in-Log in
3,1516,Annualy,Log in-Log in-Log in-Log in-Log in-Log in-Log ...
4,1516,Annualy,Log in-Log in-Log in-Log in-Log in-Log in-Log ...


In [7]:
clean = clean.groupby(['user_id', 'subscription_type'])['user_journey'].agg(lambda x: '-'.join(x)).reset_index()
clean

,user_id,subscription_type,user_journey
0,1516,Annualy,Log in-Other-Log in-Log in-Log in-Log in-Log i...
1,3395,Annualy,Log in-Log in-Sign up-Log in-Pricing-Sign up-H...
2,10107,Annualy,Resource center-Other-Resource center-Resource...
3,11145,Monthly,Log in-Log in-Homepage-Log in-Log in-Log in-Lo...
4,12400,Monthly,Log in-Log in-Log in-Log in-Sign up-Log in-Car...
...,...,...,...
1204,508638,Annualy,Other-Other-Coupon-Coupon-Coupon-Coupon-Coupon...
1205,508647,Annualy,Course certificate-Pricing-Course certificate-...
1206,508649,Monthly,Other-Other-Other-Other-Other-Other-Other-Othe...
1207,508655,Annualy,Courses-Courses-Courses-Courses-Career tracks-...


In [8]:
def remove_duplicates(string):
    pages = string.split('-')
    i = 0
    while i < len(pages)-1:
        if pages[i] == pages[i+1]: 
            pages.pop(i)
        else:
            i += 1
    string = '-'.join(pages)
    return string

In [9]:
clean['user_journey'] = clean['user_journey'].apply(remove_duplicates)

In [10]:
clean.head()

,user_id,subscription_type,user_journey
0,1516,Annualy,Log in-Other-Log in-Homepage-Log in-Sign up-Ot...
1,3395,Annualy,Log in-Sign up-Log in-Pricing-Sign up-Homepage...
2,10107,Annualy,Resource center-Other-Resource center-Homepage...
3,11145,Monthly,Log in-Homepage-Log in-Homepage-Log in-Homepag...
4,12400,Monthly,Log in-Sign up-Log in-Career tracks-Sign up-Ca...


In [11]:
clean.to_csv('cleaned data.csv')

## Data Analysis

In [12]:
def page_count(df):
    user_journey = np.array(df)
        #splitting pages
    for i in range(len(user_journey)):
        user_journey[i] = np.array(user_journey[i].split('-'))
        #creating a dictionary, with a record representing each page in user journeys
    results = {}
    results['Total pages'] = 0
    for journey in user_journey:
        for page in journey: 
            #the line retrieves the record for the page if it is present already, and initializes one for it if not
            results[page] = results.get(page, 0) + 1 
            results['Total pages'] += 1 
    #sorting the dictionary by the page count
    results = dict(sorted(results.items(), key=lambda item: item[1]))
    return results

In [13]:
def page_presence (data):
    user_journey = np.array(data)
    
    #Splitting each user journey into a set of page. Unlike lists, sets will keep on only one occurence of each page in a journey
    for i in range(len(user_journey)):
        user_journey[i] = set(user_journey[i].split('-'))

    presence = {}
    presence['Total journeys'] = len(user_journey)

    for journey in user_journey:
        for page in journey: 
            presence[page] = presence.get(page, 0) + 1 

    #Sorting the dictionary by page occurences
    presence = dict(sorted(presence.items(), key= lambda item: item[1], reverse=True)) 

    return presence

In [14]:
def avg_journey_length (data):
    user_journey = np.array(data)
    
    #Splitting journeys into separate pages 
    for i in range(len(user_journey)):
        user_journey[i] = np.array(user_journey[i].split('-'))
    
    journey_lengths = np.zeros(len(user_journey))
    
    #Getting the length of each user journey
    for i in range(len(user_journey)):
        journey_lengths[i] = len(user_journey[i])
    
    #Calculating the average journey length 
    journey_avg_length = np.sum(journey_lengths) / len(journey_lengths)

    return journey_avg_length

In [15]:
def page_sequence (data, number_of_pages=3):
    # data is the series to get checked for sequences
    # n is the number of pages to check for as its occurence a sequence
    user_journey = np.array(data)

    #Splitting journeys into separate pages 
    for i in range(len(user_journey)):
        user_journey[i] = np.array(user_journey[i].split('-'))

    sequences = {}
    for journey in user_journey: 
        flag = {}                                  #For each user journey, if a page sequence is already captured, the flag is there to ignore it
        for i in range(len(journey) - number_of_pages + 1 ): 
            page_sequence = tuple(journey[i : i + number_of_pages])
              #If the sequence is already present in the journey, skip the current iteration
            if flag.get(page_sequence, False): continue
            sequences[page_sequence] = sequences.get(page_sequence, 0) + 1
            flag[page_sequence] = True

    sequences = dict(sorted(sequences.items(), key= lambda item: item[1], reverse=True))
    return sequences

In [16]:
def page_destinations(data): 
    #Splitting journeys into separate pages 
    user_journey = [journey.split('-') for journey in np.array(data)]

    #Obtaining a unique set of the pages
    page_set = set()
    for journey in user_journey: 
        page_set.update(journey) 
    
    #Constructing a dictionary of dictionaries, where each key represent a page
    destinations = {key: dict() for key in page_set}

    #For each page in the dictionary, we count how many times it was the source for all other pages
    for journey in user_journey: 
        for i in range(len(journey)-1):
            source_page = journey[i]
            destination_page = journey[i+1]
            destinations[source_page][destination_page] = destinations[source_page].get(destination_page, 0) + 1 
    
    for key, page in destinations.items():
        destinations[key] = dict(sorted(page.items(), key = lambda item: item[1], reverse = True))
    
    return destinations

In [17]:
def limit(data, limit=10):
    # Function limits the number of records returned from a dictionary
    keys = sorted(data, key=data.get, reverse= True)[:limit]
    limited = {key: sequences[key] for key in keys}
    return limited

#### The frequency of pages

In [18]:
page_count = page_count(clean['user_journey'])
page_count

{'Blog': 25,
 'About us': 28,
 'Instructors': 53,
 'Success stories': 84,
 'Course certificate': 399,
 'Coupon': 619,
 'Resource center': 661,
 'Career track certificate': 810,
 'Other': 1010,
 'Checkout': 1280,
 'Pricing': 1369,
 'Sign up': 1568,
 'Career tracks': 1614,
 'Courses': 1654,
 'Log in': 2026,
 'Homepage': 2072,
 'Total pages': 15272}

#### How often a page contributes to a journey?

In [19]:
presence = page_presence(clean['user_journey'])
presence

{'Total journeys': 1209,
 'Checkout': 752,
 'Homepage': 725,
 'Log in': 655,
 'Sign up': 604,
 'Coupon': 531,
 'Other': 412,
 'Pricing': 402,
 'Courses': 371,
 'Career tracks': 306,
 'Career track certificate': 181,
 'Resource center': 146,
 'Course certificate': 118,
 'Success stories': 27,
 'Instructors': 25,
 'About us': 15,
 'Blog': 11}

#### Average length of journey

In [20]:
average_length = avg_journey_length(clean['user_journey'])
average_length

12.631927212572373

#### Popular page sequences?

In [21]:
sequences = page_sequence(clean.user_journey)
limit(sequences)

{('Log in', 'Homepage', 'Log in'): 305,
 ('Homepage', 'Log in', 'Checkout'): 222,
 ('Sign up', 'Homepage', 'Sign up'): 154,
 ('Career tracks', 'Courses', 'Career tracks'): 137,
 ('Career tracks', 'Homepage', 'Career tracks'): 134,
 ('Courses', 'Homepage', 'Courses'): 123,
 ('Courses', 'Career tracks', 'Courses'): 119,
 ('Log in', 'Sign up', 'Log in'): 111,
 ('Sign up', 'Courses', 'Sign up'): 106,
 ('Homepage', 'Log in', 'Homepage'): 105}

#### Page follow-ups

In [22]:
destination = pd.DataFrame(page_destinations(clean['user_journey']))
destination

,Resource center,Sign up,Instructors,Log in,Blog,Other,Career track certificate,Coupon,Success stories,About us,Courses,Pricing,Homepage,Career tracks,Checkout,Course certificate
Other,335.0,45.0,4.0,149.0,9.0,NaN,31.0,20.0,20.0,5.0,79.0,44.0,32.0,55.0,27.0,31.0
Pricing,52.0,125.0,5.0,82.0,3.0,58.0,102.0,5.0,6.0,2.0,193.0,NaN,374.0,130.0,152.0,34.0
Courses,49.0,235.0,7.0,93.0,3.0,80.0,100.0,5.0,5.0,2.0,NaN,184.0,232.0,481.0,23.0,64.0
Homepage,46.0,268.0,20.0,608.0,1.0,42.0,108.0,14.0,6.0,3.0,235.0,187.0,NaN,312.0,112.0,55.0
Log in,37.0,357.0,3.0,NaN,2.0,163.0,29.0,65.0,3.0,1.0,102.0,139.0,605.0,73.0,196.0,20.0
Career track certificate,35.0,59.0,5.0,28.0,NaN,34.0,NaN,4.0,8.0,2.0,89.0,94.0,104.0,234.0,7.0,82.0
Career tracks,34.0,195.0,1.0,57.0,2.0,53.0,200.0,5.0,12.0,NaN,529.0,120.0,297.0,NaN,25.0,31.0
Course certificate,27.0,28.0,1.0,11.0,NaN,32.0,83.0,NaN,NaN,NaN,70.0,36.0,52.0,40.0,3.0,NaN
Sign up,14.0,NaN,1.0,254.0,NaN,48.0,79.0,19.0,4.0,2.0,212.0,134.0,278.0,177.0,25.0,34.0
Success stories,9.0,2.0,NaN,7.0,1.0,22.0,10.0,1.0,NaN,2.0,4.0,7.0,4.0,12.0,1.0,NaN
